# Chapter 11: Testing with Out-of-Sample Data

In [12]:
import sys
import os
sys.path.append('/home/yutanagano/Projects/nnfs')
os.chdir('/home/yutanagano/Projects/nnfs')

In [13]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

nnfs.init()

import nnn

## Setup

In [14]:
# Create dataset
X, y = spiral_data(samples=1000, classes=3)

# Create the necessary layers
dense1 = nnn.layer.Dense(n_inputs=2,n_neurons=32,l2w=5e-4,l2b=5e-4)
relu = nnn.activation.Relu()
dropout = nnn.layer.Dropout(0.1)
dense2 = nnn.layer.Dense(n_inputs=32,n_neurons=3)
activation_loss = nnn.loss.SoftmaxWithCategoricalCrossentropy()

# Create the optimiser
optimiser = nnn.optimiser.Adam(learning_rate=0.005)

## Training

In [15]:
# Training loop
for epoch in range(10001):
    # Forward pass
    output = dense1.forward(X)
    output = relu.forward(output)
    output = dropout.forward(output)
    output = dense2.forward(output)

    # Calculate the network's current loss
    data_loss = activation_loss.forward(output, y)
    reg_loss = activation_loss.categoricalcrossentropy.regularsiation_loss(dense1) +\
        activation_loss.categoricalcrossentropy.regularsiation_loss(dense2)
    loss = data_loss + reg_loss

    # Calculate accuracy
    predictions = np.argmax(output,axis=1)
    if len(y.shape) == 2: y = np.argmax(y,axis=1)
    accuracy = np.mean(predictions == y)

    # Print accuracy
    if not epoch % 100: print(
        f"epoch: {epoch}, acc: {accuracy:.3f}, loss: {loss:.3f}, "
        f"(data_loss: {data_loss}, reg_loss: {reg_loss}), "
        f"lr: {optimiser.current_learning_rate}"
    )

    # Backward pass
    activation_loss.backward(activation_loss.outputs, y)
    dense2.backward(activation_loss.dinputs)
    dropout.backward(dense2.dinputs)
    relu.backward(dropout.dinputs)
    dense1.backward(relu.dinputs)

    # Update weights and biases
    optimiser.pre_update_params()
    optimiser.update_params(dense1)
    optimiser.update_params(dense2)
    optimiser.post_update_params()

epoch: 0, acc: 0.313, loss: 1.099, (data_loss: 1.0986073017120361, reg_loss: 2.962743164971471e-06), lr: 0.005
epoch: 100, acc: 0.484, loss: 1.007, (data_loss: 1.0040286779403687, reg_loss: 0.0029602911323308944), lr: 0.005
epoch: 200, acc: 0.558, loss: 0.930, (data_loss: 0.9237253665924072, reg_loss: 0.006308419525623322), lr: 0.005
epoch: 300, acc: 0.581, loss: 0.908, (data_loss: 0.9003919363021851, reg_loss: 0.007582645297050476), lr: 0.005
epoch: 400, acc: 0.604, loss: 0.864, (data_loss: 0.8549649119377136, reg_loss: 0.009118103623390198), lr: 0.005
epoch: 500, acc: 0.636, loss: 0.820, (data_loss: 0.8092005848884583, reg_loss: 0.010598394632339478), lr: 0.005
epoch: 600, acc: 0.628, loss: 0.825, (data_loss: 0.81386798620224, reg_loss: 0.01158379316329956), lr: 0.005
epoch: 700, acc: 0.637, loss: 0.810, (data_loss: 0.7976713180541992, reg_loss: 0.012417269706726075), lr: 0.005
epoch: 800, acc: 0.654, loss: 0.792, (data_loss: 0.7788466811180115, reg_loss: 0.01268672800064087), lr: 0.

## Evaluation

In [16]:
# Create the validation data set
X_test, y_test = spiral_data(samples=100, classes=3)

In [17]:
# Forward pass on validation set
output = dense1.forward(X_test)
output = relu.forward(output)
output = dense2.forward(output)

# Calculate validation loss
loss = activation_loss.forward(output, y_test)

In [18]:
# Calculate accuracy
predictions = np.argmax(output, axis=1)
if len(y_test.shape) == 2:
    y_test = np.argmax(y_test, axis=1)
accuracy = np.mean(predictions == y_test)

print(f"validation, acc: {accuracy:.3f}, loss: {loss:.3f}")

validation, acc: 0.737, loss: 0.703
